In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys, os, re, time
import numpy as np
import cv2
import time 
import android

haven't recieved anything new yet


In [23]:
!adb forward tcp:9999 tcp:59999

In [21]:
import SocketServer
from time import sleep
from gmail import Gmail
import datetime
import winsound, sys
import threading

global time

def beep():
    winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)

def rec():
    g = Gmail()
    g.login("blindbandit01@gmail.com","FairyTail01")
    print "Welcome"
    print "Successfully logged in"
    lastline = 0
    lastlength = len(g.inbox().mail())
    print "All old files are there"
    lasttext = False
    currenttext = False
    g.logout()
    print "haven't recieved anything new yet"
    global run
    while True:
        if not run:
            g = Gmail()
            g.login("blindbandit01@gmail.com","FairyTail01")
            a = g.inbox().mail()
            b = len(a)
            if lastlength != b:
                currenttext = True
                if b - lastlength >= 1:
                        if not run:
                            beep()
                            sleep(0.5)
                            beep()
                            sleep(0.5)
                            beep()
                        print "recieved "+ str(b - lastlength) + " new Email(s)"
                else:
                    print "NONE"
            else:
               currenttext = False
            if lasttext and not currenttext:
                print "haven't recieved anything yet"
            lasttext = currenttext
            lastlength = b
            g.logout()
            sleep(1)
        else:
            print "STOP PASSED"

            

def timer():
    global run;
    print "Timer enabled"
    while(True):
        if run:
            print "started timer"
            for a in range(0, 30, 1):
                sleep(1)
                print "seconds: " + str(a)
            run = False



if __name__ == "__main__":
    global run;
    run = False
    HOST, PORT = "", 8080
    threads = []
    t = threading.Thread(target=rec) # I KEPT THIS SO IT WOULD BE EASY JUST TO UNCOMMENT THE REST
    threads.append(t)
    t.start()



Welcome to the Email Reciever made by David
Successfully logged in
All old files are there


Exception in thread Thread-14:
Traceback (most recent call last):
  File "D:\Users\Chrys\Anaconda\lib\threading.py", line 810, in __bootstrap_inner
    self.run()
  File "D:\Users\Chrys\Anaconda\lib\threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-17-10670388b8de>", line 29, in rec
    g.login("blindbandit01@gmail.com","FairyTail01")
  File "gmail\gmail.py", line 106, in login
    raise AuthenticationError
AuthenticationError

Exception in thread Thread-16:
Traceback (most recent call last):
  File "D:\Users\Chrys\Anaconda\lib\threading.py", line 810, in __bootstrap_inner
    self.run()
  File "D:\Users\Chrys\Anaconda\lib\threading.py", line 763, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-21-e358cedb45ff>", line 29, in rec
    g.login("blindbandit01@gmail.com","FairyTail01")
  File "gmail\gmail.py", line 106, in login
    raise AuthenticationError
AuthenticationError
Exception in thread Thread-11

In [31]:
import android
import time
import gdata.spreadsheet.service
import gdata.spreadsheet.text_db
import string

droid = android.Android()

dt=100 #Time between sensing

def countdown():
    #3 Second Countdown and Vibrate
    droid.vibrate(100)
    time.sleep(1)
    droid.vibrate(100)
    time.sleep(1)
    droid.vibrate(100)#3 second countdown
    time.sleep(1)
    droid.vibrate(300)
    return

def login(myEmail='blindbandit01@gmail.com', password='FairyTail01'):    #Get Email and Password and login
    while password == '':
        email = droid.dialogGetInput("Email", "Enter the email account where the Data will be uploaded to:",myEmail).result
        password=droid.dialogGetPassword("Password", "Enter the account password where the Data will be uploaded to:").result
        try:
            client = gdata.spreadsheet.text_db.DatabaseClient()
            client.SetCredentials(username=email, password=password)
        except gdata.spreadsheet.text_db.BadCredentials:
            droid.dialogCreateAlert("Username or password incorrect.")
            droid.dialogSetPositiveButtonText("OK")
            droid.dialogShow()
            redo = droid.dialogGetResponse()
            password =''
            continue
        else:
            return client

def spreadSheetSetup(client, defaultTitle = 'Science Data'):
    versionNumber = client.GetDatabases(name=defaultTitle)
    if len(versionNumber):
        droid.dialogCreateAlert('Science Data %d' % len(versionNumber))
        droid.dialogSetPositiveButtonText("OK") #Yes option box
        droid.dialogShow()
        understood = droid.dialogGetResponse()
        spreadsheetTitle = defaultTitle+' '+str(len(versionNumber))
    else:
        spreadsheetTitle = defaultTitle
    return client.CreateDatabase(spreadsheetTitle)

def time2Sense(defaultTime=5000):
    maxTime =0
    #Enter time for sensing
    while maxTime == 0: ##How long to sense?
        try:
            maxTime=int(droid.dialogGetInput("Time", "How long to collect data (in milliseconds):",str(defaultTime)).result)
        except TypeError:
            droid.dialogCreateAlert("Please enter a numeric time")
            droid.dialogSetPositiveButtonText("OK")
            droid.dialogShow()
            droid.dialogGetResponse()
            continue
        if maxTime(30000): #30 seconds max for optimal upload
            droid.dialogCreateAlert("Warning this could take a long time to upload.")
            droid.dialogSetPositiveButtonText("I accept the risks")
            droid.dialogSetNegativeButtonText('New time')
            droid.dialogShow()
            check = droid.dialogGetResponse().result
            if check['which'] == 'positive':
                return maxTime
            else:
                maxTime = 0
                continue
        return maxTime

def doScience():
    droid.dialogCreateAlert('Are you ready', 'Click yes to begin sensing, or no to cancel')
    droid.dialogSetPositiveButtonText("Let's do science!") #Yes option box
    droid.dialogSetNegativeButtonText("Nah let's cancel") #No option box
    droid.dialogShow()
    return droid.dialogGetResponse().result

def senseAccel(endTime,dt=100):    #Sense and store accel data
    timeSensed=0
    tempScience = []
    droid.startSensingTimed(2,dt) #dt = time between sensings
    while timeSensed == endTime:
        tempScience.append(droid.sensorsReadAccelerometer().result) #add new accel data to list
        time.sleep(dt/1000.0)
        timeSensed+=dt
    #Stop Sensing
    droid.makeToast("Done")
    droid.vibrate(300)
    droid.stopSensing()
    return tempScience

def main():
    db =spreadSheetSetup(login())
    endTime = time2Sense()
    ready = doScience()
    if ready['which'] == 'positive':
        countdown()
        science = senseAccel(endTime,dt)
        table=db.CreateTable('Accelerometer',['t','x','y','z']) #Beware spreadsheet column names cannot be uppercase if you are going to add to them later http://code.google.com/p/gdata-python-client/issues/detail?id=363
        droid.dialogCreateHorizontalProgress('Progress','Uploading to Docs',len(science)*3 )
        droid.dialogShow()
        progress = 0
        time = 0
        for data in science:
            table.AddRecord({'t':str(time),'x':str(data[0]),'y':str(data[1]),'z':str(data[2])}) #AddRecord requires strings as arguements str()
            progress+=3
            time+=dt
            droid.dialogSetCurrentProgress(progress)
        droid.dialogCreateAlert('Done', 'Go check your data on Docs')
        droid.dialogSetPositiveButtonText("OK") #Yes option box
        droid.dialogShow()
        droid.dialogGetResponse()

if __name__ == "__main__":
        main()

ImportError: No module named gdata.spreadsheet.service